In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
#carrega o dataset
url = "/datacov.csv"
data = pd.read_csv(url)
#exibe as informações do dataset
data.info()
print(f"Read in {len(data)} rows")

C:\Users\Rafael\AppData\Local\Temp\ipykernel_13712\1080176038.py:3: DtypeWarning: Columns (15,59,61,62,63,91,93,122,143,171) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554328 entries, 0 to 554327
Columns: 173 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(95), int64(12), object(66)
memory usage: 731.6+ MB
Read in 554328 rows


In [3]:
#Remove colunas irrelevantes
data.drop(['DT_NASC', 'TP_IDADE','CS_ESCOL_N', 'CO_PAIS', 'DT_DIGITA', 'FNT_IN_COV', 'ID_PAIS','LOTE_2_COV', 
        'AVE_SUINO', 'POS_PCRFLU', 'POS_PCROUT', 'HISTO_VGM', 'PCR_SARS2', 'ID_REGIONA','ID_MUNICIP', 
        'ID_UNIDADE', 'ID_RG_RESI', 'ID_MN_RESI', 'ID_RG_INTE', 'ID_MN_INTE', 'COD_IDADE', 'LOTE_1_COV'], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554328 entries, 0 to 554327
Columns: 151 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(90), int64(8), object(53)
memory usage: 638.6+ MB


In [4]:
#Remoção das linhas com 3 e 9
aux = data[(data['EVOLUCAO'] == 3) | (data['EVOLUCAO'] == 9)].index
data.drop(aux, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526276 entries, 0 to 554327
Columns: 151 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(90), int64(8), object(53)
memory usage: 610.3+ MB


In [5]:
x = data
x.head()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,NU_IDADE_N,CS_GESTANT,CS_RACA,SG_UF,CO_RG_RESI,CO_MUN_RES,CS_ZONA,SURTO_SG,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,PUERPERA,FATOR_RISC,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OBES_IMC,OUT_MORBI,MORB_DESC,VACINA,DT_UT_DOSE,MAE_VAC,DT_VAC_MAE,M_AMAMENTA,DT_DOSEUNI,DT_1_DOSE,DT_2_DOSE,ANTIVIRAL,TP_ANTIVIR,OUT_ANTIV,DT_ANTIVIR,HOSPITAL,DT_INTERNA,SG_UF_INTE,CO_RG_INTE,CO_MU_INTE,UTI,DT_ENTUTI,DT_SAIDUTI,SUPORT_VEN,RAIOX_RES,RAIOX_OUT,DT_RAIOX,AMOSTRA,DT_COLETA,TP_AMOSTRA,OUT_AMOST,PCR_RESUL,DT_PCR,TP_FLU_PCR,PCR_FLUASU,FLUASU_OUT,PCR_FLUBLI,FLUBLI_OUT,PCR_VSR,PCR_PARA1,PCR_PARA2,PCR_PARA3,PCR_PARA4,PCR_ADENO,PCR_METAP,PCR_BOCA,PCR_RINO,PCR_OUTRO,DS_PCR_OUT,CLASSI_FIN,CLASSI_OUT,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PAC_COCBO,PAC_DSCBO,OUT_ANIM,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,TOMO_OUT,DT_TOMO,TP_TES_AN,DT_RES_AN,RES_AN,POS_AN_FLU,TP_FLU_AN,POS_AN_OUT,AN_SARS2,AN_VSR,AN_PARA1,AN_PARA2,AN_PARA3,AN_ADENO,AN_OUTRO,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,FAB_COV_1,FAB_COV_2,FAB_COVREF,LAB_PR_COV,LOTE_REF,DOSE_2REF,FAB_COVRF2,LOTE_REF2,TRAT_COV,TIPO_TRAT,OUT_TRAT,DT_TRT_COV
0,05/01/2022,1,02/01/2022,1,RJ,NaN,330455,7101856,M,23,6,4,RJ,NaN,330455.0,NaN,NaN,NaN,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,05/01/2022,RJ,NaN,330455.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,05/01/2022,4.0,OROFARINGE,2.0,10/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,3.0,1.0,05/01/2022,07/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,25/01/2021,22/02/2021,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04/01/2022,1,03/01/2022,1,MS,1966.0,500370,2371375,F,37,5,4,MS,1966.0,500370.0,1.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,1.0,04/01/2022,MS,1966.0,500370.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,04/01/2022,1.0,NaN,1.0,08/01/2022,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,1.0,06/01/2022,10/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,NaN,NaN,NaN,2.0,04/01/2022,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,07/06/2021,10/09/2021,NaN,87 - COVID-19 PFIZER - COMIRNATY,NaN,NaN,87 - COVID-19 PFIZER - COMIRNATY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,07/01/2022,1,04/01/2022,1,GO,1791.0,521890,5584108,F,33,5,4,GO,1791.0,521890.0,1.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,07/01/2022,GO,1791.0,521890.0,2.0,NaN,NaN,3.0,NaN,NaN,NaN,1.0,07/01/2022,1.0,NaN,2.0,11/02/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,1.0,1.0,NaN,15/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,07/01/2022,2.0,2.0,2.0,2.0,1.0,21/01/2021,17/02/2021,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07/01/2022,1,06/01/2022,1,SP,1351.0,354990,6988938,F,16,5,1,SP,1352.0,355410.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,PROVAVEL DOENCA DE LYME,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,06/01/2022,SP,1351.0,354990.0,1.

In [6]:
aux = x[(x['SG_UF_NOT'] != 'PR')].index
x.drop(aux , inplace=True)
x.drop(['SG_UF_NOT'], axis=1, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51936 entries, 13 to 554312
Columns: 150 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(90), int64(8), object(52)
memory usage: 59.8+ MB
Read in 51936 rows


In [7]:
#mantém somente diagnóstico de COVID
aux = x[(x['CLASSI_FIN'] != 5)].index
x.drop(aux , inplace=True)
x.drop(['CLASSI_FIN'], axis=1, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18222 entries, 62 to 554312
Columns: 149 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(89), int64(8), object(52)
memory usage: 20.9+ MB
Read in 18222 rows


In [8]:
x[['NOSOCOMIAL','FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'DOR_ABD',
   'FADIGA', 'PERD_OLFT', 'PERD_PALA', 'OUTRO_SIN', 'FATOR_RISC', 'PUERPERA', 'CARDIOPATI', 'HEMATOLOGI',
  'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE',
  'OUT_MORBI', 'VACINA_COV']] = x[['NOSOCOMIAL','FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'DOR_ABD',
   'FADIGA', 'PERD_OLFT', 'PERD_PALA', 'OUTRO_SIN', 'FATOR_RISC', 'PUERPERA', 'CARDIOPATI', 'HEMATOLOGI',
  'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE',
  'OUT_MORBI', 'VACINA_COV']].replace(9.0, np.nan)

In [9]:
#x.replace(9, np.nan, inplace=True)

In [10]:
x.head()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,NU_IDADE_N,CS_GESTANT,CS_RACA,SG_UF,CO_RG_RESI,CO_MUN_RES,CS_ZONA,SURTO_SG,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,PUERPERA,FATOR_RISC,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OBES_IMC,OUT_MORBI,MORB_DESC,VACINA,DT_UT_DOSE,MAE_VAC,DT_VAC_MAE,M_AMAMENTA,DT_DOSEUNI,DT_1_DOSE,DT_2_DOSE,ANTIVIRAL,TP_ANTIVIR,OUT_ANTIV,DT_ANTIVIR,HOSPITAL,DT_INTERNA,SG_UF_INTE,CO_RG_INTE,CO_MU_INTE,UTI,DT_ENTUTI,DT_SAIDUTI,SUPORT_VEN,RAIOX_RES,RAIOX_OUT,DT_RAIOX,AMOSTRA,DT_COLETA,TP_AMOSTRA,OUT_AMOST,PCR_RESUL,DT_PCR,TP_FLU_PCR,PCR_FLUASU,FLUASU_OUT,PCR_FLUBLI,FLUBLI_OUT,PCR_VSR,PCR_PARA1,PCR_PARA2,PCR_PARA3,PCR_PARA4,PCR_ADENO,PCR_METAP,PCR_BOCA,PCR_RINO,PCR_OUTRO,DS_PCR_OUT,CLASSI_OUT,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PAC_COCBO,PAC_DSCBO,OUT_ANIM,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,TOMO_OUT,DT_TOMO,TP_TES_AN,DT_RES_AN,RES_AN,POS_AN_FLU,TP_FLU_AN,POS_AN_OUT,AN_SARS2,AN_VSR,AN_PARA1,AN_PARA2,AN_PARA3,AN_ADENO,AN_OUTRO,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,FAB_COV_1,FAB_COV_2,FAB_COVREF,LAB_PR_COV,LOTE_REF,DOSE_2REF,FAB_COVRF2,LOTE_REF2,TRAT_COV,TIPO_TRAT,OUT_TRAT,DT_TRT_COV
62,25/01/2022,4,16/01/2022,3,1356.0,410690,15245,F,52,5,9,PR,1356.0,410690.0,1.0,NaN,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,"CEFALEIA, MIALGIA",2.0,1,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,"41,9",2.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,20/01/2022,PR,1356.0,410690.0,1.0,20/01/2022,09/03/2022,2.0,6.0,NaN,NaN,1.0,21/01/2022,1.0,NaN,1.0,28/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,09/03/2022,09/03/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,6.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,25/05/2021,30/08/2021,NaN,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,NaN,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,28/01/2022,4,28/01/2022,4,1369.0,411520,2586444,M,66,6,1,PR,1369.0,411520.0,1.0,NaN,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,28/01/2022,PR,1369.0,411520.0,2.0,NaN,NaN,3.0,6.0,NaN,NaN,1.0,28/01/2022,1.0,NaN,1.0,28/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,31/01/2022,17/03/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,1.0,NaN,28/01/2022,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,19/04/2021,19/07/2021,NaN,FIOCRUZ,FIOCRUZ,NaN,FIOCRUZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,03/02/2022,5,28/01/2022,4,1356.0,410690,15644,M,70,6,1,PR,1356.0,411915.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,CONGESTAO NASAL,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,LLC,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,02/02/2022,PR,1356.0,410690.0,1.0,03/02/2022,06/02/2022,1.0,6.0,NaN,NaN,1.0,02/02/2022,1.0,NaN,1.0,05/02/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,06/02/2022,09/02/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,30/03/2021,23/04/2021,05/11/2021,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,87 - COVID-19 PFIZER - COMIRNATY,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,FH8025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,07/02/2022,6,04/02/2022,5,1374.0,411460,4053907,F,78,5,1,PR,1374.0,411460.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [11]:
#remoção das colunas com 50% de dados faltantes
x.dropna(axis=1, how='any', thresh=9111, subset=None, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18222 entries, 62 to 554312
Data columns (total 61 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DT_NOTIFIC  18222 non-null  object 
 1   SEM_NOT     18222 non-null  int64  
 2   DT_SIN_PRI  18222 non-null  object 
 3   SEM_PRI     18222 non-null  int64  
 4   CO_REGIONA  18222 non-null  float64
 5   CO_MUN_NOT  18222 non-null  int64  
 6   CO_UNI_NOT  18222 non-null  int64  
 7   CS_SEXO     18222 non-null  object 
 8   NU_IDADE_N  18222 non-null  int64  
 9   CS_GESTANT  18222 non-null  int64  
 10  CS_RACA     18222 non-null  int64  
 11  SG_UF       18206 non-null  object 
 12  CO_RG_RESI  18198 non-null  float64
 13  CO_MUN_RES  18206 non-null  float64
 14  CS_ZONA     16164 non-null  float64
 15  NOSOCOMIAL  16459 non-null  float64
 16  FEBRE       14970 non-null  float64
 17  TOSSE       15632 non-null  float64
 18  GARGANTA    13596 non-null  float64
 19  DISPNEIA    15309 non-n

In [12]:
#Remoção das linhas com 50% de dados faltantes
x.dropna(axis=0, thresh=38, subset=None, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17892 entries, 62 to 554312
Data columns (total 61 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DT_NOTIFIC  17892 non-null  object 
 1   SEM_NOT     17892 non-null  int64  
 2   DT_SIN_PRI  17892 non-null  object 
 3   SEM_PRI     17892 non-null  int64  
 4   CO_REGIONA  17892 non-null  float64
 5   CO_MUN_NOT  17892 non-null  int64  
 6   CO_UNI_NOT  17892 non-null  int64  
 7   CS_SEXO     17892 non-null  object 
 8   NU_IDADE_N  17892 non-null  int64  
 9   CS_GESTANT  17892 non-null  int64  
 10  CS_RACA     17892 non-null  int64  
 11  SG_UF       17876 non-null  object 
 12  CO_RG_RESI  17868 non-null  float64
 13  CO_MUN_RES  17876 non-null  float64
 14  CS_ZONA     15937 non-null  float64
 15  NOSOCOMIAL  16275 non-null  float64
 16  FEBRE       14885 non-null  float64
 17  TOSSE       15518 non-null  float64
 18  GARGANTA    13544 non-null  float64
 19  DISPNEIA    15235 non-n

In [13]:
x.head()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,NU_IDADE_N,CS_GESTANT,CS_RACA,SG_UF,CO_RG_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,FATOR_RISC,CARDIOPATI,DIABETES,OUT_MORBI,VACINA,ANTIVIRAL,HOSPITAL,DT_INTERNA,SG_UF_INTE,CO_RG_INTE,CO_MU_INTE,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,DT_COLETA,TP_AMOSTRA,PCR_RESUL,DT_PCR,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,RES_AN,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,FAB_COV_1,FAB_COV_2,LAB_PR_COV
62,25/01/2022,4,16/01/2022,3,1356.0,410690,15245,F,52,5,9,PR,1356.0,410690.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,2.0,9.0,2.0,1.0,20/01/2022,PR,1356.0,410690.0,1.0,2.0,6.0,1.0,21/01/2022,1.0,1.0,28/01/2022,1.0,2.0,09/03/2022,09/03/2022,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,25/05/2021,30/08/2021,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD
80,28/01/2022,4,28/01/2022,4,1369.0,411520,2586444,M,66,6,1,PR,1369.0,411520.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,1.0,2.0,2.0,2.0,2.0,1.0,28/01/2022,PR,1369.0,411520.0,2.0,3.0,6.0,1.0,28/01/2022,1.0,1.0,28/01/2022,1.0,1.0,31/01/2022,17/03/2022,2.0,2.0,2.0,2.0,1.0,5.0,2.0,1.0,19/04/2021,19/07/2021,FIOCRUZ,FIOCRUZ,FIOCRUZ
99,03/02/2022,5,28/01/2022,4,1356.0,410690,15644,M,70,6,1,PR,1356.0,411915.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1,NaN,NaN,1.0,9.0,2.0,1.0,02/02/2022,PR,1356.0,410690.0,1.0,1.0,6.0,1.0,02/02/2022,1.0,1.0,05/02/2022,1.0,2.0,06/02/2022,09/02/2022,NaN,NaN,NaN,NaN,NaN,4.0,2.0,1.0,30/03/2021,23/04/2021,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC
175,25/03/2022,12,16/02/2022,7,1356.0,410690,15563,F,6,6,9,PR,1356.0,410690.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1,2.0,2.0,2.0,9.0,2.0,1.0,18/02/2022,PR,1356.0,410690.0,1.0,1.0,5.0,1.0,18/02/2022,1.0,1.0,18/02/2022,1.0,1.0,18/03/2022,26/04/2022,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,21/06/2022,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC
234,28/05/2022,21,26/05/2022,21,1374.0,412770,4056698,F,25,5,1,PR,1374.0,412770.0,1.0,NaN,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2,NaN,NaN,NaN,NaN,2.0,1.0,28/05/2022,PR,1374.0,412770.0,2.0,3.0,6.0,1.0,28/05/2022,4.0,1.0,06/06/2022,1.0,1.0,01/06/2022,06/06/2022,2.0,2.0,2.0,2.0,6.0,5.0,2.0,1.0,19/08/2021,21/09/2021,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC


In [14]:
x.columns

Index(['DT_NOTIFIC', 'SEM_NOT', 'DT_SIN_PRI', 'SEM_PRI', 'CO_REGIONA',
       'CO_MUN_NOT', 'CO_UNI_NOT', 'CS_SEXO', 'NU_IDADE_N', 'CS_GESTANT',
       'CS_RACA', 'SG_UF', 'CO_RG_RESI', 'CO_MUN_RES', 'CS_ZONA', 'NOSOCOMIAL',
       'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO',
       'DIARREIA', 'VOMITO', 'OUTRO_SIN', 'FATOR_RISC', 'CARDIOPATI',
       'DIABETES', 'OUT_MORBI', 'VACINA', 'ANTIVIRAL', 'HOSPITAL',
       'DT_INTERNA', 'SG_UF_INTE', 'CO_RG_INTE', 'CO_MU_INTE', 'UTI',
       'SUPORT_VEN', 'RAIOX_RES', 'AMOSTRA', 'DT_COLETA', 'TP_AMOSTRA',
       'PCR_RESUL', 'DT_PCR', 'CRITERIO', 'EVOLUCAO', 'DT_EVOLUCA',
       'DT_ENCERRA', 'DOR_ABD', 'FADIGA', 'PERD_OLFT', 'PERD_PALA', 'TOMO_RES',
       'RES_AN', 'ESTRANG', 'VACINA_COV', 'DOSE_1_COV', 'DOSE_2_COV',
       'FAB_COV_1', 'FAB_COV_2', 'LAB_PR_COV'],
      dtype='object')

In [15]:
from datetime import datetime

x = x.drop(['SG_UF','SG_UF_INTE'], axis=1)
x['DT_NOTIFIC1'] = pd.to_datetime(x['DT_NOTIFIC'], format='%d/%m/%Y', errors='coerce').apply(datetime.toordinal)
x['DT_SIN_PRI1'] = pd.to_datetime(x['DT_SIN_PRI'], ).apply(datetime.toordinal)
x['DT_INTERNA1'] = pd.to_datetime(x['DT_INTERNA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_COLETA1'] = pd.to_datetime(x['DT_COLETA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_PCR1'] = pd.to_datetime(x['DT_PCR'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_EVOLUCA1'] = pd.to_datetime(x['DT_EVOLUCA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_ENCERRA1'] = pd.to_datetime(x['DT_ENCERRA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DOSE_1_COV1'] = pd.to_datetime(x['DOSE_1_COV'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DOSE_2_COV1'] = pd.to_datetime(x['DOSE_2_COV'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
#x.head()
x = x.drop(['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_INTERNA', 'DT_COLETA', 'DT_PCR', 'DT_EVOLUCA', 'DT_ENCERRA', 'DOSE_1_COV', 'DOSE_2_COV'], axis=1)
x["CS_SEXO"] = x["CS_SEXO"].replace({"F": 0, "M": 1})
x["FAB_COV_1"] = x["FAB_COV_1"].replace({"85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD": 85, "87 - COVID-19 PFIZER - COMIRNATY": 87,
                                         "86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC": 86, "89 - COVID-19 ASTRAZENECA - CHADOX1-S": 89,
                                         "88 - COVID-19 JANSSEN - AD26.COV2.S": 88, "FIOCRUZ": 85.1,
                                         "COVID-19-COVISHIELD-OXFORD/FIOCRUZ": 90, "BUTANTAN": 91,
                                         "OXFORD/FIOCRUZ ASTRAZENICA": 92, "CORONAVAC": 93,
                                         "COVID-19-CORONAVAC-SINOVAC/BUTANTAN": 94,
                                         "ASTRAZENECA": 95, "BUTANTA": 91, "SINOVAC":96, "PFIZER":97, "OXFORD": 98,
                                         "JANSSEN":99, "99 - COVID-19 PEDIATRICA - PFIZER COMIRNATY":100,
                                         "99 - COVID-19 PEDIATRICA - PFIZER COMIRNATY": 101, "JANSE": 99,
                                         "BUTANTAN/3ª DPSE 06/10/21 PFIZER": 102, "PFEZIR": 97, "CORONAVAC/BUTANTAN": 103, "CORONAVAC - BUTANTAN": 103,
                                         "OXFORD/ASTRAZENECA" : 104, "ASTRAZENECA AB": 95, "98 - COVID-19 SINOVAC - CORONAVAC":96,
                                         "OXFORD//3 DOSE LOTE:20/04/2021 PFIZER": 105, "VACINA COVID-19 BUTANTAN": 91})
x["FAB_COV_2"] = x["FAB_COV_2"].replace({"85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD": 85, "87 - COVID-19 PFIZER - COMIRNATY": 87,
                                         "86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC": 86, "89 - COVID-19 ASTRAZENECA - CHADOX1-S": 89,
                                         "88 - COVID-19 JANSSEN - AD26.COV2.S": 88, "FIOCRUZ": 85.1,
                                         "COVID-19-COVISHIELD-OXFORD/FIOCRUZ": 90, "BUTANTAN": 91,
                                         "OXFORD/FIOCRUZ ASTRAZENICA": 92, "CORONAVAC": 93,
                                         "COVID-19-CORONAVAC-SINOVAC/BUTANTAN": 94,
                                         "ASTRAZENECA": 95, "BUTANTA": 91, "SINOVAC":96, "PFIZER":97, "OXFORD": 98,
                                         "JANSSEN":99, "99 - COVID-19 PEDIATRICA - PFIZER COMIRNATY":100,
                                         "99 - COVID-19 PEDIATRICA - PFIZER COMIRNATY": 101, "JANSE": 99,
                                         "BUTANTAN/3ª DPSE 06/10/21 PFIZER": 102, "PFEZIR": 97, "CORONAVAC/BUTANTAN": 103, "CORONAVAC - BUTANTAN": 103,
                                         "OXFORD/ASTRAZENECA" : 104, "ASTRAZENECA AB": 95, "98 - COVID-19 SINOVAC - CORONAVAC":96,
                                         "OXFORD//3 DOSE LOTE:20/04/2021 PFIZER": 105, "VACINA COVID-19 BUTANTAN": 91})
x["LAB_PR_COV"] = x["LAB_PR_COV"].replace({"85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD": 85, "87 - COVID-19 PFIZER - COMIRNATY": 87,
                                         "86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC": 86, "89 - COVID-19 ASTRAZENECA - CHADOX1-S": 89,
                                         "88 - COVID-19 JANSSEN - AD26.COV2.S": 88, "FIOCRUZ": 85.1,
                                         "COVID-19-COVISHIELD-OXFORD/FIOCRUZ": 90, "BUTANTAN": 91,
                                         "OXFORD/FIOCRUZ ASTRAZENICA": 92, "CORONAVAC": 93,
                                         "COVID-19-CORONAVAC-SINOVAC/BUTANTAN": 94,
                                         "ASTRAZENECA": 95, "BUTANTA": 91, "SINOVAC":96, "PFIZER":97, "OXFORD": 98,
                                         "JANSSEN":99, "99 - COVID-19 PEDIATRICA - PFIZER COMIRNATY":100,
                                         "99 - COVID-19 PEDIATRICA - PFIZER COMIRNATY": 101, "JANSE": 99,
                                         "BUTANTAN/3ª DPSE 06/10/21 PFIZER": 102, "PFEZIR": 97, "CORONAVAC/BUTANTAN": 103, "CORONAVAC - BUTANTAN": 103,
                                         "OXFORD/ASTRAZENECA" : 104, "ASTRAZENECA AB": 95, "98 - COVID-19 SINOVAC - CORONAVAC":96,
                                         "OXFORD//3 DOSE LOTE:20/04/2021 PFIZER": 105, "VACINA COVID-19 BUTANTAN": 91})
x = x.drop(["LAB_PR_COV","FAB_COV_2","FAB_COV_1"], axis= 1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/01/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '28/01/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/02/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '26/05/2022' in DD/MM/YYYY format. Provide format or specif

In [16]:
from sklearn.impute import SimpleImputer

dadosnumericos = x[['SEM_NOT', 'SEM_PRI', 'NU_IDADE_N', 'CS_GESTANT', 'DT_NOTIFIC1', 'DT_SIN_PRI1', 'DT_INTERNA1', 'DT_COLETA1',
                   'DT_PCR1', 'DT_EVOLUCA1', 'DT_ENCERRA1', 'DOSE_1_COV1', 'DOSE_2_COV1']]
x = x.drop(['SEM_NOT', 'SEM_PRI', 'NU_IDADE_N', 'CS_GESTANT', 'DT_NOTIFIC1', 'DT_SIN_PRI1', 'DT_INTERNA1', 'DT_COLETA1',
                   'DT_PCR1', 'DT_EVOLUCA1', 'DT_ENCERRA1', 'DOSE_1_COV1', 'DOSE_2_COV1'], axis=1)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
nome = dadosnumericos.columns
imputer = imputer.fit(dadosnumericos)
dadosnumericos = imputer.transform(dadosnumericos)
dadosnumericos = pd.DataFrame(dadosnumericos)
dadosnumericos.columns = nome


imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
nomes = x.columns
imputer = imputer.fit(x)
x = imputer.transform(x)
x = pd.DataFrame(x)
x.columns = nomes
x.loc[x['VACINA_COV'] == 2.0, ['DOSE_1_COV', 'DOSE_2_COV', 'FAB_COV_1','FAB_COV_2', 'LAB_PR_COV']] = np.nan
x = pd.concat([x, dadosnumericos], axis=1)

#--------------------------------------------------------------------------------------------------------------------------
#y.head()
#for linha in x:
#        if linha['VACINA_COV'] == 2..0:
#            for i in range(len(row)):
#                if i not in ['DOSE_1_COV', 'DOSE_2_COV', 'FAB_COV_1','FAB_COV_2', 'LAB_PR_COV']:
#                    imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#
#--------------------------------------------------------------------------------------------------------------------------
#if (x['VACINA_COV'] == 2.0):
#    x = pd.DataFrame(x)
#    x_drop = x.drop(axis=0, inplace=True)
#    imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#    imputer = imputer.fit(x)
#    x = imputer.transform(x)
#    x = pd.DataFrame(x)
#    x = pd.concat([x, x_drop], axis=1)
#else:
#    imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#    imputer = imputer.fit(x)    
#    x = imputer.transform(x)
#    x = pd.DataFrame(x)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)


In [19]:
y = x['EVOLUCAO']
x = x.drop(columns= 'EVOLUCAO')
pd.set_option('display.max_rows', None)
x.iloc[0:5, :]

,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,CS_RACA,CO_RG_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,FATOR_RISC,CARDIOPATI,DIABETES,OUT_MORBI,VACINA,ANTIVIRAL,HOSPITAL,CO_RG_INTE,CO_MU_INTE,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,TP_AMOSTRA,PCR_RESUL,CRITERIO,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,RES_AN,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,FAB_COV_1,FAB_COV_2,LAB_PR_COV,SEM_NOT,SEM_PRI,NU_IDADE_N,CS_GESTANT,DT_NOTIFIC1,DT_SIN_PRI1,DT_INTERNA1,DT_COLETA1,DT_PCR1,DT_EVOLUCA1,DT_ENCERRA1,DOSE_1_COV1,DOSE_2_COV1
0,1356.0,410690.0,15245.0,0.0,9.0,1356.0,410690.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,9.0,2.0,1.0,1356.0,410690.0,1.0,2.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,4.0,3.0,52.0,5.0,738180.0,738171.0,738175.0,738176.0,738183.0,738401.0,738401.0,737935.0,738032.0
1,1369.0,411520.0,2586444.0,1.0,1.0,1369.0,411520.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1369.0,411520.0,2.0,3.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,5.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,66.0,6.0,738183.0,738183.0,738183.0,738183.0,738183.0,1.0,1.0,737899.0,737990.0
2,1356.0,410690.0,15644.0,1.0,1.0,1356.0,411915.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,9.0,2.0,1.0,1356.0,410690.0,1.0,1.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,70.0,6.0,738189.0,738183.0,738188.0,738188.0,738191.0,738308.0,738400.0,737879.0,737903.0
3,1356.0,410690.0,15563.0,0.0,9.0,1356.0,410690.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,9.0,2.0,1.0,1356.0,410690.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,12.0,7.0,6.0,6.0,738239.0,738202.0,738204.0,738204.0,738204.0,1.0,1.0,738327.0,1.0
4,1374.0,412770.0,4056698.0,0.0,1.0,1374.0,412770.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,9.0,2.0,1.0,1374.0,412770.0,2.0,3.0,6.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,5.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,21.0,21.0,25.0,5.0,738303.0,738301.0,738303.0,738303.0,738312.0,738161.0,738312.0,738021.0,738054.0


In [21]:
x.replace(np.nan,0, inplace=True)

In [22]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=5, random_state=43)

score = cross_val_score(clf, x, y, cv=10)
print(f"Accuracy of floresta aleátoria Classifier is:{score}")
print('Média: {:.2} | Desvio: {:.2}'.format(np.mean(score), np.std(score)))

Accuracy of floresta aleátoria Classifier is:[0.81787709 0.83072626 0.83286752 0.82504192 0.82895472 0.82057015
 0.83119061 0.82560089 0.83174958 0.82895472]
Média: 0.83 | Desvio: 0.0047
